In [27]:
import zipfile
import requests
from collections import Counter
import re
import math

x = requests.get("http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("spam.zip", "wb") as f:
  f.write(x.content)

zipfile.ZipFile("spam.zip").extractall("./")

data = []
with open("SMSSpamCollection.txt", "r") as f:
  for line in f:
    data.append(line.strip().split("\t"))

train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train = data[:train_size]
test = data[train_size:]

count = Counter([s for s, c in train])
prior_spam = count['spam'] / (count['spam'] + count['ham'])
prior_ham = count['ham'] / (count['spam'] + count['ham'])

num_spam_words = sum(len(re.findall("[0-9a-z_]+", c.lower())) for s,c in train if s == "spam")
num_ham_words = sum(len(re.findall("[0-9a-z_]+", c.lower())) for s,c in train if s != "spam")

spams = [c for s,c in train if s == "spam"]
hams = [c for s,c in train if s == "ham"]

spam_words = Counter([w for txt in spams for w in re.findall("[0-9a-z_]+", txt.lower())])
ham_words = Counter([w for txt in hams for w in re.findall("[0-9a-z_]+", txt.lower())])

def predict(text, k=0.5):
  words = re.findall("[0-9a-z_]+", text.lower())
  denom_spam = num_spam_words + num_unique_words * k
  denom_ham = num_ham_words + num_unique_words * k

  spam_score = math.log(prior_spam)
  ham_score = math.log(prior_ham)

  for w in words:
    spam_score += math.log((spam_words[w] + k)/denom_spam)
    ham_score += math.log((ham_words[w] + k)/denom_ham)

  return spam_score > ham_score

true_positive = 0
true_negative = 0
false_positive = 0
false_negative = 0

for s, c in test:
  pred = predict(c)
  if pred and s == 'spam':
    true_positive += 1
  elif pred and s != 'spam':
    false_positive += 1
  elif not pred and s == 'spam':
    false_negative += 1
  else:
    true_negative += 1

print("accuracy: ", (true_positive + true_negative) / (true_positive + false_positive + true_negative + false_negative))
print("precision: ", true_positive/(true_positive + false_positive))
print("recall: ", true_positive / (true_positive + false_negative))
print("f1: ", true_positive / (true_positive + (false_positive + false_negative)/2) )


accuracy:  0.9847533632286996
precision:  0.9923076923076923
recall:  0.8896551724137931
f1:  0.9381818181818182
